### Trabalho Final - Aprendizado de Máquina
- Vinícius Braga Freire - 2020054889

#### 0. Introdução

Este projeto...

In [1]:
# !pip install lightgbm

#### 1. Processamento de dados

Assim, precisamos primeiro extrair e tratar os dados relevantes da nossa base de dados de forma que consigamos aprender sob esses.

##### 1.1 Projetos

O primeiro passo será extrair dados que nos permitam observar as interações entre funcionários e\nportante\ncomo as equipes são estruturadas. Dada a chegada de novos dados do projeto IMAGINE (um acrônimo interno da empresa DEXCO), foi reformulada a maneira como iremos modelar e interpretar times neste projeto.

O projeto IMAGINE permite que funcionários façam sugestões de projetos internos que sejam inovadores. Funcionários que apresentem ideias boas têm seus projetos aceitos e implantados e, portanto, recebem um bônus em remuneração por tal feito.

Desta forma, nosso dado irá nos informar o status (aceito, rejeitado, em processo de implantação, em espera de avaliação) de projetos que foram sugeridos e por quais funcionários.

Observe que mais de um funcionário podem trabalhar juntos para sugerirem ideias de projetos. É a partir deste fato que iremos construir um novo dado seguindo as seguintes regras:

- Em projetos aceitos e que foram sugeridos por mais de 1 pessoa, devemos sinalizar positivamente que as pessoas envolvidas `DEVEM` trabalhar em equipe. Assim
essas pessoas 2 a 2 (cada par possível dentre esse conjunto de pessoas) tem um aval positivo\ndado que o projeto sugerido por eles foi aceito como uma boa ideia.
- Em projetos rejeitados e que foram sugeridos por mais de 1 pessoa, devemos sinalizar negativamente que as pessoas envolvidas `NÃO DEVEM` trabalhar em equipe.

In [2]:
import pandas as pd
import numpy as np

projetos_df = pd.read_csv("./data/in/imagine.csv", encoding="utf-8", sep=",")
print(projetos_df.info())
projetos_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58730 entries, 0 to 58729
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  58716 non-null  object
 1   Matricula             58730 non-null  int64 
 2   status                58730 non-null  object
 3   local                 58729 non-null  object
 4   area                  58729 non-null  object
 5   data                  58730 non-null  object
 6   suggestionsRegarding  27664 non-null  object
dtypes: int64(1), object(6)
memory usage: 3.1+ MB
None


,name,Matricula,status,local,area,data,suggestionsRegarding
0,Custometro,10077261,Fila de execução,ADM CENTRAL,CSS-01 - PRESIDENCIA CONSELHO ADM - EC,2/23/2022,NaN
1,otmizar a temperatura do oléo térmico no desca...,10023570,Fila de execução,FÁB ITAPETININGA,DXCAVAPO61 - Caldeira de Vapor,7/5/2022,Produtividade
2,limpeza de sensor de nivel,10023570,Concluída,FÁB ITAPETININGA,DXCAVAPO61 - Caldeira de Vapor,10/20/2022,NaN
3,Segurança sempre!,10023570,Fila de execução,FÁB ITAPETININGA,MDP - Caldeira,11/9/2021,Segurança
4,Em busca da economia do BPF!,10023570,Fila de execução,FÁB ITAPETININGA,MII-02 - CALDEIRA E UILIDADES - IT,8/22/2019,Redução de Custos


Observe que temos alguns dados faltantes, principalmente na coluna `Name`, que será importante para nós agruparmos os dados por projeto para acharmos as equipes. Assim, vamos limpar os dados.

In [3]:
projetos = projetos_df[['name', 'Matricula', 'status']]
# Remove null names since it's impossible to group the data
projetos = projetos.dropna(subset=['name'])
projetos

,name,Matricula,status
0,Custometro,10077261,Fila de execução
1,otmizar a temperatura do oléo térmico no desca...,10023570,Fila de execução
2,limpeza de sensor de nivel,10023570,Concluída
3,Segurança sempre!,10023570,Fila de execução
4,Em busca da economia do BPF!,10023570,Fila de execução
...,...,...,...
58725,PONTO DE COLETA SELETIVA!,10001462,Não qualificada
58726,limpou a agua,10001131,Não qualificada
58727,Nacionalização de segmento do desfibrador,10001131,Fila de execução
58728,Redução Mão de Obra Terceirizada na Colômbia,27534,Fila de execução


Vamos codificar o `status` em números para que possamos processar os dados posteriormente de maneira mais rápida. O mapeamento será o seguinte:
- 'Cancelada': 0
- 'Concluída': 1
- 'Não qualificada': 2
- 'Em execução': 3
- 'Fila de execução': 4
- 'Stand By': 5
- 'Pendente de patente': 6
- 'Aguardando qualificação' = 'Esperando Calificación': 7

In [4]:
NUM_TO_STATS = ['Cancelada',
                'Concluída',
                'Não qualificada',
                'Em execução',
                'Fila de execução', 
                'Stand By',
                'Pendente de patente',
                'Aguardando qualificação']
STATS_TO_NUM = {'Cancelada': 0,
                'Concluída': 1,
                'Não qualificada': 2,   # Decided to put non qualified after concluded and not after
                'Em execução': 3,       # canceled considering that non qualified means that the work
                'Fila de execução': 4,  # was not bad, but with any invalid requirement
                'Stand By': 5, 
                'Pendente de patente': 6,
                'Aguardando qualificação': 7,
                'Esperando Calificación': 7,}

print(projetos['status'].unique())
# Transform the data
projetos['status'] =  projetos['status'].apply(lambda row: STATS_TO_NUM[row]).values
projetos

['Fila de execução' 'Concluída' 'Aguardando qualificação' 'Em execução'
 'Stand By' 'Não qualificada' 'Cancelada' 'Pendente de patente'
 'Esperando Calificación']


,name,Matricula,status
0,Custometro,10077261,4
1,otmizar a temperatura do oléo térmico no desca...,10023570,4
2,limpeza de sensor de nivel,10023570,1
3,Segurança sempre!,10023570,4
4,Em busca da economia do BPF!,10023570,4
...,...,...,...
58725,PONTO DE COLETA SELETIVA!,10001462,2
58726,limpou a agua,10001131,2
58727,Nacionalização de segmento do desfibrador,10001131,4
58728,Redução Mão de Obra Terceirizada na Colômbia,27534,4


Agora vamos agrupar as pessoas que trabalharam no mesmo projeto, filtrando assim projetos que só tiveram 1 pessoa.

In [5]:
# Group people that worked in the same project (and in projects with more than 1 person)
mask = projetos.groupby('name').count()['Matricula'] > 1
# Removes projects that only one person work on
map = {}
for name, valid in mask.items(): map[name] = valid
full_mask = projetos['name'].apply(lambda x: map[x] if x in map else print('ERROR AT' + x))
proj_gt_1 = projetos[full_mask]

# Avoids repeated person in same project
proj_gt_1 = proj_gt_1.drop_duplicates()
proj_gt_1

,name,Matricula,status
0,Custometro,10077261,4
1,otmizar a temperatura do oléo térmico no desca...,10023570,4
9,Marcação de peças,10077515,7
10,GABARITOS PARA TODAS AS BANCAS,10079166,4
11,reaproveitamento térmico para climatização amb...,10073270,7
...,...,...,...
58725,PONTO DE COLETA SELETIVA!,10001462,2
58726,limpou a agua,10001131,2
58727,Nacionalização de segmento do desfibrador,10001131,4
58728,Redução Mão de Obra Terceirizada na Colômbia,27534,4


Agora iremos garatir que uma pessoa só apareça uma vez por projeto afim de remover redundâncias. Além disso, iremos tratar os casos onde um mesmo projeto está atribuido à diferentes status.

Neste ultimo caso iremos operar da seguinte maneira: para remover a duplicidade de dados iremos estabelecer uma ordem de prioridade dos status. Os status de numero menor (estabelecidos há 2 celulas acima) terão prioridade sobre os de maior número. Assim, o status de um grupo será igual ao menor número de status encontrado no mesmo. Isto é importante, pois caso um mesmo grupo tenha os estados `Cancelada` e `Concluída`, queremos que o projeto seja considerado como cancelado.

In [6]:
# Group the projects by name, gattering the teams
gp = proj_gt_1.groupby('name')
df = pd.DataFrame(columns=['name', 'Matricula', 'status'])

# For each project, removes internal redundancy and put it in a final data frame
for name, workers in gp:
    # Tries to find the lines which there are people with more than 1 status (diverging info)
    stats_unique = workers['status'].unique()
    if len(stats_unique) != 1:
        # Gets the new status
        min_stat = min(stats_unique)
        workers['status'] = min_stat

    # Tries to find the lines which a same person appears more than 1 time
    n_unique = len(workers['Matricula'].unique())
    if n_unique != workers.shape[0]:
        workers = workers.drop_duplicates()

    # Make sure projects that, after drop of duplicates, has more than 1 person working
    if workers.shape[0] != 1:
        df=pd.concat([df, workers], ignore_index=True)

# Final data frame
df

,name,Matricula,status
0,ALINHAR AS ESTEIRAS TRANSPORTADORAS COM A NR1...,10089243,7
1,ALINHAR AS ESTEIRAS TRANSPORTADORAS COM A NR1...,10081841,7
2,Alteração layout Linear máquina 67 e 69,10092156,1
3,Alteração layout Linear máquina 67 e 69,10092076,1
4,Alteração layout Linear máquina 67 e 69,10091449,1
...,...,...,...
24193,Ônibus Escola - Viação Caierias,10075320,4
24194,Ônibus Escola - Viação Caierias,10072520,4
24195,Ônibus Escola - Viação Caierias,10061987,4
24196,índice para resina,10088493,1


##### 1.2 Interações

Agora queremos construir uma representação das interações entre funcionários. Como o objetivo do projeto é prever se uma dada dupla deveria ou não trabalhar junta (e assim ir acoplando pessoas de forma a gerar times), devemos ter uma métrica para utilizarmos como label e que nos indicará se duas pessoas devem ou não trabalhar juntas. 

Assim, usaremos o status do projeto para se gerar estas labels. Pessoas que pertencem à projetos com label 1 (concluída) `DEVEM` trabalhar juntas, enquanto pessoas com label 0 (cancelada) `NÃO DEVEM` trabalhar. 

Dessas relações obteremos um grafo. Para isso, faremos:

In [7]:
import networkx as nx

projects = df[df['status'] <= 1].groupby('name')
G = nx.Graph()

for name, workers in projects:
    n_workers = workers.shape[0]
    status = 1 if workers.iloc[0]['status'] == 1 else -1
    
    # Cria uma aresta (ou aumenta o peso de aresta existente) entre cada uma das pessoas que participaram dessa reunião
    for i in range(n_workers):
        _from = int(workers.iloc[i]['Matricula'])
        for j in range(i + 1, n_workers):
            to = int(workers.iloc[j]['Matricula'])
            
            # If the edge has already been created
            if G.has_edge(_from, to):
                G[_from][to]['weight'] += status
                G[_from][to]['n_projs'] += 1
            # Initialize an edge
            else:
                G.add_edge(_from, to, weight=status, n_projs=1)
                
# Normalize the edges values
for edge in G.edges():
    G[edge[0]][edge[1]]["weight"] /= G[edge[0]][edge[1]]['n_projs']

Note que acima nos normalizamos os dados. Isto foi necessário, pois ao se explorar os dados foi notado que muitas duplas de funcionários possuiam um alto valor atribuído à eficiência da dupla. A array a seguir é um exemplo de valores únicos encontrados em todo o grafo.

```python
array([  3,   1,   2,   9,   4,   0,  -1,  -3,  -2,   5,   8,   7,  -4,
        10,   6,  16,  18,  14,  15,  11,  72,  57,  -7,  12,  13,  -8,
        -6,  -9, -10,  22,  17,  49,  47,  19])
```

Acima notamos que tem alguns valores bastante altos, ou seja, tiveram duplas que teoricamente funcionaram muito bem. Vamos observar o porque desses valores que, de primeira vez, parecem anômalos.

Por exemplo, a dupla `(10046705, 10077310)` obteve um score total de 72, ou seja, eles funcionam muito bem juntos. Se observarmos abaixo, veremos que eles participaram de um total de 199 projetos e isto que justifica este valor tão alto.

In [8]:
print(f"Esta dupla participou de {df[df['Matricula'] == 10046705]['name'].unique().shape[0]} projetos em conjunto.")

Esta dupla participou de 199 projetos em conjunto.


Portanto, normalizamos este dado ao fazer $ 72 / 199 $, afinal pessoas que trabalham 199 vezes e tiveram maus resultados algumas vezes não tem mais sincronia do que uma dupla que trabalhou 3 vezes e as 3 vezes deram certo.

#### 2. Treinamento de modelo

##### 2.1. NBNE

Agora vamos trabalhar no treinamento dos modelos. Primeiro iremos aproveitar que conseguimos processar e transformar as interações entre funcionários em um grafo no passo anterior e utilizaremos dessa estrutura como uma nova forma de representação.

Assim, iremos utilizar do algoritmo NBNE (Neighbor Based Node Embeddings), um algoritmo não supervisionado que é capaz de aprender a codificar nós de um grafo em um vetor. Assim, o que este algoritmo faz é aprender a codificar um nó em um embedding. Cada embedding possui um certo número de dimensões que representam abstratamente características do nó, ou seja, são diretamente afetados por como é a vizinhança do mesmo e que tipo de relação ele possuia com seus vizinhos (as aresta entre eles tinham um valor positivo - pessoas que deveriam trabalhar junto - ou negativo - pessoas que não deveriam trabalhar junto).

Assim, agora iremos obter a representação dos nossos nós, ou seja, de cada funcionário. Inicialmente vamos fazer cada embedding ter 128 dimensões.

In [9]:
from nbne import train_model
from os.path import exists

NBNE_DIM = 128
NBNE_FILE = f"./data/out/nbne-{NBNE_DIM}.out"

# Just verify if the file isn't already instantiated (avoid repeated work)
if not exists(NBNE_FILE):
    train_model(graph=G, num_permutations=5, embedding_dimension=128, output_file=NBNE_FILE)

Esta operação nos garantiu um arquivo que armazena os embeddings de todos os nossos funcionários. Este dado será o nosso dado de entrada para os futuros algoritmos de predição. Assim, vamos recuperar este dado e armazená-lo em um formato que nos permita acessar rapidamente cada funcionário e sua representação.

In [29]:
from typing import Dict

# A hash map mapping workers registration ID to their embedding vector
funcs : Dict[int, np.array] = {}

with open(NBNE_FILE, mode="r") as file:
    lines = file.readlines()
    
    # For each worker
    for line in lines[1:]:
        data = line.split(" ")
        # Each worker, mapped by its ID stores its embedding
        funcs[int(data[0])] = np.array(data[1:]).astype(np.float64)

# Stores all the workers registration IDs
funcsID = list(funcs.keys())

Agora iremos reconstruir o grafo em formato de dataframe (que será utilizado na parte de link prediction). Este dataframe terá as seguintes colunas:
- $ 2 \times NBNE_{DIM} $ colunas, onde $ NBNE_{DIM} $ é o número de dimensões do embedding. Essas colunas representarão a concatenação do embedding de 2 funcionários que já trabalharam juntos, ou seja, que possuem uma aresta no grafo.
- Mais uma coluna, que será a label do nosso dado. Nesta coluna estará o valor do peso da aresta entre esses 2 nós, ou seja, o quão bom foi o desempenho entre 2 funcionários.

Iremos adicionar 2 colunas extras, porém estas não serão utilizadas no treinamento, mas sim apenas para fatores de debugging. Essas duas colunas irão armazenas o identificador dos funcionários que estão sendo representados por estes embeddings.

In [11]:
RELATIONS_FILE = f"./data/out/relations-{NBNE_DIM}.csv"

def writeArrInCsv(arr: np.array, file, sep: str):
    for el in arr:
        file.write(f"{el}{sep}")

# If the file doesn't exists, creates it
if not exists(RELATIONS_FILE):
    with open(RELATIONS_FILE, "w+") as file:
        # Writes the column names of the data
        file.write("id1,id2,")
        for i in range(NBNE_DIM): file.write(f"a{i},")
        for i in range(NBNE_DIM): file.write(f"b{i},")
        file.write("quality\n")

        # For each worker interaction in the graph
        for edge in G.edges():
            file.write(f"{edge[0]},{edge[1]},")
            writeArrInCsv(funcs[edge[0]], file, sep=",")
            writeArrInCsv(funcs[edge[1]], file, sep=",")
            file.write(f"{G[edge[0]][edge[1]]['weight']}\n")

# 
relations = pd.read_csv(RELATIONS_FILE, sep=",")
relations.head(2)

,id1,id2,a0,a1,a2,a3,a4,a5,a6,a7,...,b119,b120,b121,b122,b123,b124,b125,b126,b127,quality
0,10092156,10092076,-0.059191,-0.050889,-0.222815,-0.030142,0.363358,0.070741,0.101035,0.126196,...,0.342778,-0.048073,-0.541260,-0.365752,0.275575,0.087210,-0.258118,0.004593,-0.010365,1.0
1,10092156,10091449,-0.059191,-0.050889,-0.222815,-0.030142,0.363358,0.070741,0.101035,0.126196,...,0.439382,-0.062711,-0.698245,-0.486721,0.337008,0.120705,-0.317609,0.013949,0.001175,1.0


Vamos observar então os dados que foram gerados. Vamos ver os dados de qualidade das duplas de funcionários. 

Após a normalização vemos que agora os dados estão mais consistentes e em um range reduzido, o que facilitará a vida do algorítmo. Note que 1 é o valor máximo para o quanto dois funcionários deveriam trabalhar e -1 para o quanto eles não deveriam trabalhar juntos. Note então que valores mais próximos de 0 indicam um alto grau de incerteza, ou seja, a predição é tão boa quanto jogar uma moeda ao ar para decidir se uma dupla deveria ou não trabalhar. 

In [12]:
relations[['id1', 'id2', 'quality']]['quality'].unique()

array([ 1.        ,  0.33333333,  0.        ,  0.2       ,  0.5       ,
        0.66666667, -1.        , -0.33333333,  0.6       ,  0.8       ,
        0.63636364,  0.81818182, -0.42857143,  0.88888889, -0.6       ,
       -0.5       ,  0.97297297, -0.77777778,  0.55555556, -0.83333333,
        0.42857143])

##### 2.2. Visualização de dados

Apenas uma pequena seção com funções que irão nos auxiliar a visualizar os dados e os treinos.

In [13]:
import matplotlib.pyplot as plt

def plotTrainVal(evals, subplot, metric):
    subplot.set_title(f"Train with '{metric}' metric")
    subplot.set_xlabel("Number of iterations")
    subplot.set_ylabel("Error")
    subplot.yaxis.grid(color='#777', alpha=0.45,linestyle='dashed')
    subplot.xaxis.grid(color='#777', alpha=0.45,linestyle='dashed')
    
    for key in evals:
        hist = evals[key][metric]
        subplot.plot(range(len(hist)), hist, label=f"{'train' if key == 'training' else 'test'}")
    subplot.legend()

def plotTrain(evals, title: str, evalMetrics: list[str]):
    outer = plt.figure(figsize=(9, 8))
    outer.suptitle(f"{title}", fontsize=16)
    subplots = outer.subplots(2, 1)
    # outer.tight_layout()
    plt.subplots_adjust(hspace=0.3)
        
    i = 0
    for metric in evalMetrics:
        # Plots each validation
        plotTrainVal(evals, subplots[i], metric)
        i += 1
        
    plt.plot()

##### 2.3. Link prediction

Com todos esses dados em mãos, podemos finalmente começarmos a treinar modelos que fazem predições a respeito de algo em função de nossos funcionários. Primeiramente iremos treinar um modelo que aprenda a nos dizer o quão bom será o desempenho de uma dupla caso ambos trabalhem juntos.

Para este caso iremos dar o nome de `Link Prediction`, pois o que este algoritmo fará é prever se, dado dois nós (dois funcionários) qual será o peso da aresta que os liga. Concretamente, o que este algoritmo está fazendo é fazendo uma previsão de como será o `link`, a aresta, entre esses nós. Já abstratamente, o que este algoritmo fará é prever o quaão bom será o desempenho de duas pessoas caso trabalhem juntas.

Deste modo, se torna claro que precisaremos utilizar de um algoritmo de regressão para fazer tais predições.

In [32]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import joblib
import random

# Function that trains an lightgbm regressor for link prediction. It takes:
# 
def trainLinkPrediction(n_estimators: int, metrics: list[str], file: str, l_rate = 0.05):
    # Defines the model
    model : lgb.LGBMRegressor = lgb.LGBMRegressor(
        learning_rate = l_rate,
        n_estimators = n_estimators,
    )
    # Split the data into train and test
    x_train, x_test, y_train, y_test = train_test_split(
        relations.drop(['id1', 'id2', 'quality'], axis=1), 
        relations['quality'], 
        test_size=0.25
    )

    evals = {}
    # Trains the model
    model.fit(x_train, 
              y_train,
              eval_set = [(x_train, y_train), (x_test, y_test)],
              eval_metric = metrics,
              callbacks = [lgb.record_evaluation(evals), 
                           lgb.early_stopping(50)],
              )
    joblib.dump(model, file)
    return model, evals

# Loads a pre-trained model from disk. The file path should be specified
def loadModel(file: str):
    return joblib.load(file)

In [15]:
EVAL_METRICS = ['l1', 'l2']
NUM_TREES = NBNE_DIM * 2
MODEL_NAME = f'./data/models/nbne{NBNE_DIM}-tree{NBNE_DIM * 2}.pkl'

model, evals = trainLinkPrediction(NUM_TREES, EVAL_METRICS, MODEL_NAME)

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[256]	training's l1: 0.0616198	training's l2: 0.0162929	valid_1's l1: 0.149676	valid_1's l2: 0.105765


In [ ]:
plotTrain(evals, "Treino 128 embedding", EVAL_METRICS)

Vamos definir uma função que faz uma predição de desempenho dado uma certa dupla.

In [109]:
# Predicts a pair quality by using a certain model. The pairs don't need to be an 
#  existing pair (don't need to be instantiated in the relations data frame)
def predictPairQuality(id1: int, id2: int, model):
    embedding1 = funcs[id1].reshape(1, -1)
    embedding2 = funcs[id2].reshape(1, -1)
    # Predicts based on this data
    return model.predict(np.concatenate((embedding1, embedding2), axis=1))[0]

# Predicts the quality of an random, but existing, pair quality in a way we can 
#  compare the predicted value and the original value.
def predictRandomExistingPairQuality():
    example = relations.sample(n=1)
    prediction = predictPairQuality(example['id1'].iloc[0], example['id2'].iloc[0], model)
    print(f"Label: {example['quality'].iloc[0]} | Predicted: {prediction}")
    

In [103]:
# Gets 2 different random workers
rndWorkers = random.sample(funcsID, 2)
# Predicts
predictPairQuality(rndWorkers[0], rndWorkers[1], model)

0.959711970308019

In [147]:
predictRandomExistingPairQuality()

Label: 1.0 | Predicted: 0.9902985905745003


#### 3. Resultado e validação de modelo

##### 1.2. Infos Pessoais